# Group THE BOYS 1D ML project
1. Loy Pek Yong 1004475
2. Lim Yongqing 1005955


# Question 1

## Importing libraries + reading data + preprocessing

In [1]:
# import libraries
import pandas as pd
import numpy as np
print(np.__version__)
print(pd.__version__)

with open('./ES/train', 'r', encoding = 'utf8') as f:
    r1 = [lines.rstrip("\n") for lines in f.readlines()]
ES_train_tup = [ (lines.rsplit(" ", 1)[0], lines.rsplit(" ", 1)[1]) if lines != "" else ("null", "null") for lines in r1]

with open('./ES/dev.in', 'r', encoding = 'utf8') as f:
    r2 = [lines.rstrip("\n") for lines in f.readlines()]
ES_dev_in_tup = [ lines if lines != "" else "null" for lines in r2]

with open('./ES/dev.out', 'r', encoding = 'utf8') as f:
    r3 = [lines.rstrip("\n") for lines in f.readlines()]
ES_dev_out_tup = [ (lines.rsplit(" ", 1)[0], lines.rsplit(" ", 1)[1]) if lines != "" else ("null", "null") for lines in r3]



with open('./RU/train', 'r', encoding = 'utf8') as f:
    r4 = [lines.rstrip("\n") for lines in f.readlines()]
RU_train_tup = [ (lines.rsplit(" ", 1)[0], lines.rsplit(" ", 1)[1]) if lines != "" else ("null", "null") for lines in r4]

with open('./RU/dev.in', 'r', encoding = 'utf8') as f:
    r5 = [lines.rstrip("\n") for lines in f.readlines()]
RU_dev_in_tup = [ lines if lines != "" else "null" for lines in r5]

with open('./RU/dev.out', 'r', encoding = 'utf8') as f:
    r6 = [lines.rstrip("\n") for lines in f.readlines()]
RU_dev_out_tup = [ (lines.rsplit(" ", 1)[0], lines.rsplit(" ", 1)[1]) if lines != "" else ("null", "null") for lines in r6]


1.23.5
1.5.3


## Changing to dataframe

In [2]:
ES_train = pd.DataFrame(data = ES_train_tup, columns = ["word", "label"])
ES_dev_in = pd.DataFrame(data = ES_dev_in_tup, columns = ["word"])
ES_dev_out = pd.DataFrame(data = ES_dev_out_tup, columns = ["word", "label"])

RU_train = pd.DataFrame(data = RU_train_tup, columns = ["word", "label"])
RU_dev_in = pd.DataFrame(data = RU_dev_in_tup, columns = ["word"])
RU_dev_out = pd.DataFrame(data = RU_dev_out_tup, columns = ["word", "label"])

# Part 1 of q1

## function that returns the emission parameters with input of training set

In [3]:
def train_estimate_emission_parameters(train):
    # word_column = train["word"]
    # label_column = train["label"]

    label_count = {}
    emission_count = {}

    for row in range(train.shape[0]):
        word = train.iloc[row, 0]
        label = train.iloc[row, 1]
        if label not in label_count:
            label_count[label] = 1
        else:
            label_count[label] += 1

        if (word, label) not in emission_count:
            emission_count[(word, label)] = 1
        else:
            emission_count[(word, label)] += 1

    emission_parameter = {}
    for word, label in emission_count:
        emission_parameter[(word, label)] = emission_count[(word, label)] / label_count[label]

    return emission_parameter, emission_count, label_count


# Part 2 of q1

## function that returns the emission parameters with input set

In [4]:
def upgraded_estimate_emission_parameters(test_set, emission_parameter, emission_counts, label_counts):
    # predict the label of the test set
    k = 0
    special_word = '#UNK#'
    test_copy = test_set.copy()
    #train word list is all the first argument in keys of emission counts
    train_word_list= [word for word, label in emission_counts.keys()]
    for i in range(len(test_copy["word"])):
        word = test_copy["word"][i]
        if word not in train_word_list:
            test_copy["word"][i] = special_word
            k+=1

    for word in test_copy["word"]:
        for label in label_counts:
            if word == special_word:
                emission_parameter[(word, label)] = k / (label_counts[label] + k)
            elif ( (word, label) in emission_parameter ):
                emission_parameter[(word, label)] = emission_counts[(word, label)] / (label_counts[label] + k)
                
    # print(emission_parameter)        
    return emission_parameter

# Part 3 of q1

In [5]:
def sentiment_analysis(training_set, test_set):
    emission_parameter, emission_count, label_count = train_estimate_emission_parameters(training_set)
    emission_parameter = upgraded_estimate_emission_parameters(test_set, emission_parameter, emission_count, label_count)
    test_set["label"] = ""

    #label equals to argmax of emission parameter
    for row in range(len(test_set)):
        word = test_set.iloc[row]["word"]
        max_val = 0
        max_label = ""
        train_word_list = [word for word, label in emission_count.keys()]
        
        for label in label_count:
            if (word, label) in emission_parameter:
                if emission_parameter[(word, label)] > max_val:
                    max_val = emission_parameter[(word, label)]
                    max_label = label
                    
            else:
                if word not in train_word_list:
                    # get max label from word #UNK#
                    if ("#UNK#", label) in emission_parameter:
                        if emission_parameter[("#UNK#", label)] > max_val:
                            max_val = emission_parameter[("#UNK#", label)]
                            max_label = label
        test_set.loc[row, "label"] = max_label
                    
    return test_set


## Predicting + saving to csv

In [6]:
ES_dev_p1_out = sentiment_analysis(ES_train, ES_dev_in)
ES_dev_p1_out_r = ES_dev_p1_out.replace('null', pd.NA)
ES_dev_p1_out_r.to_csv("./ES/dev.p1.out", sep=" ", header=None, index=False, encoding="utf-8", quoting=3)
    
RU_dev_p1_out = sentiment_analysis(RU_train, RU_dev_in)
RU_dev_p1_out_r = RU_dev_p1_out.replace('null', pd.NA)
RU_dev_p1_out_r.to_csv("./RU/dev.p1.out", sep=" ", header=None, index=False, encoding="utf-8", quoting=3)


# Scoring for part 1

In [7]:
import subprocess

command = ['python', r'.\ES\evalResult.py', r'.\ES\dev.out', r'.\ES\dev.p1.out']
result = subprocess.run(command, stdout=subprocess.PIPE)
print("ES result is:")
print(result.stdout.decode())

command = ['python', r'.\RU\evalResult.py', r'.\RU\dev.out', r'.\RU\dev.p1.out']
result = subprocess.run(command, stdout=subprocess.PIPE)
print("RU result is:")
print(result.stdout.decode())


ES result is:

#Entity in gold data: 229
#Entity in prediction: 1144

#Correct Entity : 181
Entity  precision: 0.1582
Entity  recall: 0.7904
Entity  F: 0.2637

#Correct Sentiment : 120
Sentiment  precision: 0.1049
Sentiment  recall: 0.5240
Sentiment  F: 0.1748

RU result is:

#Entity in gold data: 389
#Entity in prediction: 1467

#Correct Entity : 288
Entity  precision: 0.1963
Entity  recall: 0.7404
Entity  F: 0.3103

#Correct Sentiment : 174
Sentiment  precision: 0.1186
Sentiment  recall: 0.4473
Sentiment  F: 0.1875



# 

# Question 2

## Reading data + preprocessing

In [8]:
with open('./ES/train', 'r', encoding = 'utf8') as f:
    file = f.readlines()
EStrain = [lines.rstrip("\n") for lines in file]
EStrain_tup = [ (lines.rsplit(" ", 1)[0], lines.rsplit(" ", 1)[1]) if lines != "" else "STOP" for lines in EStrain ]

with open('./RU/train', 'r', encoding = 'utf8') as f:
    file = f.readlines()
RUtrain = [lines.rstrip("\n") for lines in file]
RUtrain_tup = [ (lines.rsplit(" ", 1)[0], lines.rsplit(" ", 1)[1]) if lines != "" else "STOP" for lines in RUtrain ]

# Part 1 of q2

## Creating reference transition dictionary (all transitions)

In [9]:
transit_dict = {
    ("O", "O"): 0,
    ("O", "B-positive"):0,
    ("O", "B-negative"):0,
    ("O", "B-neutral"):0,
    ("O", "I-positive"):0,
    ("O", "I-negative"):0,
    ("O", "I-neutral"):0,
    ("O", "START"):0,
    ("O", "STOP"):0,
    
    ("B-positive", "O"):0,
    ("B-positive", "B-positive"):0,
    ("B-positive", "B-negative"):0,
    ("B-positive", "B-neutral"):0,
    ("B-positive", "I-positive"):0,
    ("B-positive", "I-negative"):0,
    ("B-positive", "I-neutral"):0,
    ("B-positive", "START"):0,
    ("B-positive", "STOP"):0,
    
    
    ("B-negative", "O"):0,
    ("B-negative", "B-positive"):0,
    ("B-negative", "B-negative"):0,
    ("B-negative", "B-neutral"):0,
    ("B-negative", "I-positive"):0,
    ("B-negative", "I-negative"):0,
    ("B-negative", "I-neutral"):0,
    ("B-negative", "START"):0,
    ("B-negative", "STOP"):0,
    
    ("B-neutral", "O"):0,
    ("B-neutral", "B-positive"):0,
    ("B-neutral", "B-negative"):0,
    ("B-neutral", "B-neutral"):0,
    ("B-neutral", "I-positive"):0,
    ("B-neutral", "I-negative"):0,
    ("B-neutral", "I-neutral"):0,
    ("B-neutral", "START"):0,
    ("B-neutral", "STOP"):0,
    
    ("I-positive", "O"):0,
    ("I-positive", "B-positive"):0,
    ("I-positive", "B-negative"):0,
    ("I-positive", "B-neutral"):0,
    ("I-positive", "I-positive"):0,
    ("I-positive", "I-negative"):0,
    ("I-positive", "I-neutral"):0,
    ("I-positive", "START"):0,
    ("I-positive", "STOP"):0,
    
    ("I-negative", "O"):0,
    ("I-negative", "B-positive"):0,
    ("I-negative", "B-negative"):0,
    ("I-negative", "B-neutral"):0,
    ("I-negative", "I-positive"):0,
    ("I-negative", "I-negative"):0,
    ("I-negative", "I-neutral"):0,
    ("I-negative", "START"):0,
    ("I-negative", "STOP"):0,
    
    ("I-neutral", "O"):0,
    ("I-neutral", "B-positive"):0,
    ("I-neutral", "B-negative"):0,
    ("I-neutral", "B-neutral"):0,
    ("I-neutral", "I-positive"):0,
    ("I-neutral", "I-negative"):0,
    ("I-neutral", "I-neutral"):0,
    ("I-neutral", "START"):0,
    ("I-neutral", "STOP"):0,
    
    ("START", "O"):0,
    ("START", "B-positive"):0,
    ("START", "B-negative"):0,
    ("START", "B-neutral"):0,
    ("START", "I-positive"):0,
    ("START", "I-negative"):0,
    ("START", "I-neutral"):0,
    ("START", "START"):0,
    ("START", "STOP"):0,
    
    ("STOP", "O"):0,
    ("STOP", "B-positive"):0,
    ("STOP", "B-negative"):0,
    ("STOP", "B-neutral"):0,
    ("STOP", "I-positive"):0,
    ("STOP", "I-negative"):0,
    ("STOP", "I-neutral"):0,
    ("STOP", "START"):0,
    ("STOP", "STOP"):0,
               
}

## Function to calculate number of u (init state) to v (next state) transitions

In [10]:
"""
Assumptions made: 

1. Everything is in tuple pairs except the word "STOP" which signifies the end of a review
2. Transition probabilities from "STOP" to any other states are 0 but still included in the dictionary as required by question



Variables:
1. u --> initial state (e.g "O", "B-neutral", "I-positive" etc.)
2. v --> next state (e.g "O", "B-neutral", "I-positive" etc.)
3. hasbreak --> signifies the end of a review (boolean)

"""

def calculate_transition(data, transit_dict):
    updated_dict = transit_dict.copy()
    hasbreak = True
    for word in data:
        if ( hasbreak == True ):
            u = "START"
            
        if ( word != "STOP" and (hasbreak == True) ):
            v = word[1]
            updated_dict[(u, v)] += 1
            hasbreak = False
            
        elif ( word != "STOP" and (hasbreak == False) ):
            u = v
            v = word[1]
            updated_dict[(u, v)] += 1
        
        elif ( word == "STOP" ):
            u = v
            v = "STOP"
            updated_dict[(u, v)] += 1
            hasbreak = True
            
    return updated_dict

## Update for ES and RU

In [11]:
ES_updated_tdict = calculate_transition(EStrain_tup, transit_dict)
RU_updated_tdict = calculate_transition(RUtrain_tup, transit_dict)

## Function to convert above calculation to probability

In [12]:
def transition_param(updated_tdict):
    all_states = ['O', 'B-positive', 'B-negative', 'B-neutral', 'I-positive', 'I-negative', 'I-neutral', 'START', 'STOP']
    q_dict = {}
    temp_sum = 0
    for state in all_states:
        total = sum(value for key, value in updated_tdict.items() if key[1] == state)
        for key, value in updated_tdict.items():
            if ( (key[1] == state) and ( total != 0) ):
                q_dict[f"{state}|{key[0]}"] = value/total
            elif ( (key[1] == state) and ( total == 0) ):
                q_dict[f"{state}|{key[0]}"] = 0
    return q_dict

## *Below is the final parameters to be used*

*Format is v|u (next state given current state)*

In [13]:
ES_transit = transition_param(ES_updated_tdict)
RU_transit = transition_param(RU_updated_tdict)

# Getting emission params

In [14]:
ES_emission_parameter, ES_emission_count, ES_label_count = train_estimate_emission_parameters(ES_train)
ES_emission = upgraded_estimate_emission_parameters(ES_dev_in, ES_emission_parameter, ES_emission_count, ES_label_count)

RU_emission_parameter, RU_emission_count, RU_label_count = train_estimate_emission_parameters(RU_train)
RU_emission = upgraded_estimate_emission_parameters(RU_dev_in, RU_emission_parameter, RU_emission_count, RU_label_count)

# Read dev.in files

In [15]:
with open('./ES/dev.in', 'r', encoding = 'utf8') as f:
    file = f.readlines()
ESdev = [lines.rstrip("\n") for lines in file]
ESdev_ls = [ lines if lines != "" else "STOP" for lines in ESdev ]

with open('./RU/dev.in', 'r', encoding = 'utf8') as f:
    file = f.readlines()
RUdev = [lines.rstrip("\n") for lines in file]
RUdev_ls = [ lines if lines != "" else "STOP" for lines in RUdev ]

# Restructuring dev.in to list of sequences

In [16]:
def to_ls(dev_in_ls, transit_param):
    sequence_list = []
    sequence_xi = []
    for word in dev_in_ls:
        if ( word != "STOP"):
            sequence_xi.append(word)
        else:
            sequence_list.append(sequence_xi.copy())
            sequence_xi = []
    return sequence_list

In [17]:
ES_seq_list = to_ls(ESdev_ls, ES_transit)
RU_seq_list = to_ls(RUdev_ls, RU_transit)

## Based on previous state, get the possible next states based on the current word

In [18]:
def get_possible_states(prev_state, desired_word, emit, transit):
    filtered_keys = [key for key in emit.keys() if (key[0] == desired_word)]
    signal = False
    if ( len(filtered_keys) == 0 ):
        desired_pattern = f"|{prev_state}"
        unk = [(key, value) for key, value in transit.items() if ( key.endswith(desired_pattern) and not key.startswith("STOP") and not key.startswith("START") and value!=0 )]
        filtered_keys = [(desired_word, item[0].split('|')[0]) for item in unk]
        signal = True
    possible_states = [key[1] for key in filtered_keys]
    return possible_states, signal

# Viterbi

In [19]:
def viterbi(sequence, emit, transit, u, v, index, dt = {}):
    u = v
    dt[(0,v)] = 0
    max_u_ls = []
    possible_states = ["START"]
    word = ""
    for i in range(len(sequence)):
        save_state = possible_states
        word = sequence[index]
        signal = False
        possible_states, signal = get_possible_states(save_state[0], word, emit, transit)
        for u in save_state:
            for v in possible_states:
                a = np.log(transit[f"{v}|{u}"])
                if signal == True:
                    b = np.log(emit[('#UNK#', v)])
                else:
                    b = np.log(emit[(word, v)])
                max_u_ls.append(dt[(index, u)] + a + b)
            for j in range(len(possible_states)):
                key = ( index+1, possible_states[j])
                if key not in dt:
                    dt[key] = max_u_ls[j]
                elif dt[key] < max_u_ls[j]:
                    dt[key] = max_u_ls[j]
            max_u_ls = max_u_ls[len(possible_states):]
        max_u_ls = []
        index += 1
    dct = dt.copy()
    return dct

# Prediction function

In [20]:
def predict(emit, transit, seq_list):
    dt = {}
    pred = []
    for i in seq_list:
        temp = viterbi(i, emit, transit, "START", "START", 0, {})
        pred.append(temp)
    return pred

# Predict

#### Note: Might have divide by zero error but it's fine

In [21]:
ES_pred = predict(ES_emission, ES_transit, ES_seq_list)
RU_pred = predict(RU_emission, RU_transit, RU_seq_list)

C:\Users\limyo\AppData\Local\Temp\ipykernel_33580\4078816607.py:14: RuntimeWarning: divide by zero encountered in log
  a = np.log(transit[f"{v}|{u}"])


## For each duplicated index, look for the one with highest log value a.k.a highest prob

In [22]:
def processing_pred(pred):
    pred_ls = []
    grouped_dict = {}
    for seq in pred:
        for key, value in seq.items():
            var = key[0]
            if var in range(0, len(seq)):
                if var not in grouped_dict or value > grouped_dict[var][1]:
                    grouped_dict[var] = (key, value)

        pred_ls.append([item[0] for item in grouped_dict.values()])
        grouped_dict = {}
    return pred_ls

In [23]:
ES_proc = processing_pred(ES_pred)
RU_proc = processing_pred(RU_pred)

# Convert to df

In [24]:
def consolidate(pred_res, seq_list):
    res = pd.DataFrame(columns=['word', 'label'])
    for seq_i in range(len(seq_list)):
        labels = [label for _, label in pred_res[seq_i][1:]]
        df = pd.DataFrame({
            'word': seq_list[seq_i],
            'label': labels
        })
        df.loc[len(df)] = ''
        res = pd.concat([res, df])
    return res

In [25]:
ES_pred_q2 = consolidate(ES_proc, ES_seq_list)
RU_pred_q2 = consolidate(RU_proc, RU_seq_list)

# Saving to csv

In [26]:
ES_pred_q2.to_csv("./ES/dev.p2.out", sep=" ", header=None, index=False, encoding="utf-8", quoting=3)
RU_pred_q2.to_csv("./RU/dev.p2.out", sep=" ", header=None, index=False, encoding="utf-8", quoting=3)

# Scoring for q2

In [27]:
import subprocess

command2 = ['python', r'.\ES\evalResult.py', r'.\ES\dev.out', r'.\ES\dev.p2.out']
result2 = subprocess.run(command2, stdout=subprocess.PIPE)
print("ES result is:")
print(result2.stdout.decode())

command2 = ['python', r'.\RU\evalResult.py', r'.\RU\dev.out', r'.\RU\dev.p2.out']
result2 = subprocess.run(command2, stdout=subprocess.PIPE)
print("RU result is:")
print(result2.stdout.decode())


ES result is:

#Entity in gold data: 229
#Entity in prediction: 1014

#Correct Entity : 139
Entity  precision: 0.1371
Entity  recall: 0.6070
Entity  F: 0.2237

#Correct Sentiment : 86
Sentiment  precision: 0.0848
Sentiment  recall: 0.3755
Sentiment  F: 0.1384

RU result is:

#Entity in gold data: 389
#Entity in prediction: 1392

#Correct Entity : 234
Entity  precision: 0.1681
Entity  recall: 0.6015
Entity  F: 0.2628

#Correct Sentiment : 150
Sentiment  precision: 0.1078
Sentiment  recall: 0.3856
Sentiment  F: 0.1684

